In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import glob
import os

In [13]:
tmp = pd.read_csv('Predicted Labels/2021-12-23.csv')
tmp.head()

,time,lat,lon,SQRT_SEA_DEM_LAT,WSPD,WDIR,TMP,TX,TN,TP,...,AAI,CLOUD,CO,HCHO,NO2,O3,SO2,NDVI,Predicted_Label,Score
0,2021-12-23,23.385,105.315,1.303528,1.793588,201.18706,11.164371,16.528004,7.531985,0.0,...,-0.911479,0.403582,0.03648,0.000160,0.000015,0.115603,-0.000248,0.372199,Good,0.9673
1,2021-12-23,23.385,105.325,1.303528,1.793588,201.18706,11.164371,16.528004,7.531985,0.0,...,-0.910195,0.403582,0.03648,0.000146,0.000013,0.115533,-0.000246,0.456459,Good,0.9622
2,2021-12-23,23.375,105.305,1.303528,1.598616,198.96070,11.031870,15.719995,7.699991,0.0,...,-0.898209,0.381615,0.03648,0.000151,0.000014,0.115468,-0.000172,0.362741,Good,0.8722
3,2021-12-23,23.375,105.315,1.303528,1.598616,198.96071,11.031870,15.719995,7.699991,0.0,...,-0.884911,0.694282,0.03648,0.000141,0.000013,0.115332,-0.000095,0.577915,Good,0.9479
4,2021-12-23,23.375,105.325,1.303528,1.598616,198.96070,11.031870,15.719995,7.699991,0.0,...,-0.871614,0.518107,0.03648,0.000131,0.000012,0.115196,-0.000019,0.677869,Good,0.9131


In [25]:
# Ánh xạ nhãn AQI thành màu
aqi_label_to_color = {
    'Good': '#00e400',
    'Moderate': '#ffff00',
    'Unhealthy for Sensitive': '#ff7e00',
    'Unhealthy': '#ff0000',
    'Very Unhealthy': '#8f3f97',
    'Hazardous': '#7e0023'
}

# Đường dẫn tới thư mục chứa CSV
csv_folder = "Predicted Labels" 
csv_files = glob.glob(os.path.join(csv_folder, "*.csv"))

output_folder = "AQI Map Images"
os.makedirs(output_folder, exist_ok=True)  # Tạo thư mục images nếu chưa tồn tại

for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    df = df.dropna(subset=["lat", "lon", "Predicted_Label"])

    lats = df["lat"].values
    lons = df["lon"].values
    labels = df["Predicted_Label"].values

    # Lấy màu tương ứng với nhãn
    colors = [aqi_label_to_color.get(label, "#000000") for label in labels]  # mặc định là đen nếu không khớp

    # Vẽ bản đồ với Cartopy
    fig = plt.figure(figsize=(10, 8))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_extent([102, 110, 20, 24], crs=ccrs.PlateCarree())  # Giới hạn miền Bắc Việt Nam

    ax.add_feature(cfeature.BORDERS, linestyle=':')
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.LAND, facecolor='lightgray')
    ax.add_feature(cfeature.OCEAN, facecolor='lightblue')
    ax.add_feature(cfeature.LAKES, facecolor='lightblue')
    ax.add_feature(cfeature.RIVERS, edgecolor='blue')

    # Vẽ điểm AQI
    ax.scatter(lons, lats, c=colors, s=1, linewidth=0, transform=ccrs.PlateCarree())

    # Vẽ chú thích màu theo nhãn
    for label, color in aqi_label_to_color.items():
        ax.scatter([], [], c=color, label=label, s=30)

    ax.legend(title="AQI Level", loc="upper right")
    ax.set_title(f"Air Quality in Northern Vietnam - {os.path.basename(csv_file)}")
    # plt.show()

    # Lưu plot vào thư mục images
    output_filename = os.path.join(output_folder, f"{os.path.splitext(os.path.basename(csv_file))[0]}.png")
    plt.savefig(output_filename, bbox_inches='tight', dpi=300)
    plt.close(fig)  # Đóng figure để tiết kiệm bộ nhớ